In [2]:
from datasets import load_dataset
from collections import defaultdict
ds = load_dataset('hotpotqa/hotpot_qa', 'fullwiki', trust_remote_code=True)

/home/junseokkim00/anaconda3/envs/rp2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ds_train_rm_distract
import random

k=100
indices = random.sample(range(len(ds['train'])), k)
test_ds = ds['train'].select(indices)


In [4]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from dotenv import find_dotenv, load_dotenv
import os

load_dotenv(find_dotenv())
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model='llama3-8b-8192')

In [17]:
human_msg="""You are an assistant for question decomposing. Decompose the given a complex question into a sub question that is simple. Try to minimize the number of sub questions. Also return your sub questions in a strict format of a single python list containing strings with sub question.

Question: {question}

Sub questions:
"""
total_cnt=0
cnt=0
pred_og=[]
labels_og=[]
for data in test_ds:
    question, answer, sentences = data['question'], data['answer'], data['context']['sentences']
    print(data['supporting_facts']['title'])
    print(f"{total_cnt+1}/{len(test_ds)} Question: {question}")
    context=""
    # for sent_ls in sentences:
    #     context+=" ".join(sent_ls)
    #     context+='\n\n'
    chat_prompt = ChatPromptTemplate.from_messages([
        ("user", human_msg)
    ])
    chain = chat_prompt | llm
    output = chain.invoke({
        'question': question,
    }).content
    output = output[output.index('['):output.index(']')]
    print(output)
    raise

# print(f"exact_match: {cnt} / {total_cnt}")
    
    


['Where Rainbows End', 'Cecelia Ahern']
1/100 Question: The author of "Where Rainbows End" was born in what city?
['What is the name of the book?', 'Who is the author of the book?', 'In what city was the author born?'


RuntimeError: No active exception to reraise